# EB Dataset

The EB data set was collected from IC50, toxicity to environmental bacteria (U.S. EPA MICROTOX test)

reference: https://setac.onlinelibrary.wiley.com/doi/10.1002/etc.5620130617

https://www.ncbi.nlm.nih.gov/pubmed/12627631

https://setac.onlinelibrary.wiley.com/doi/full/10.1002/etc.5620181116


endpoint values are -log10 of Conc.(mg/kg)


## Generate rdkit continuous descriptors, splitting dataset, and descriptor preprocessing

In [1]:
from rdkit import Chem
import pandas as pd
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
import os
currentDirectory = os.getcwd()
d = os.path.join(currentDirectory, "Datasets", "EB_0801.csv")
dataset = pd.read_csv(d, index_col = 0)
molecules = [Chem.MolFromSmiles(mol) for mol in dataset.SMILES]

calculator = MoleculeDescriptors.MolecularDescriptorCalculator([desc[0] for desc in Descriptors.descList])
X = pd.DataFrame([list(calculator.CalcDescriptors(mol)) for mol in molecules],
                     index=dataset.index,
                     columns=list(calculator.GetDescriptorNames()))

train_set_X, test_set_X = train_test_split(X, test_size=0.2, random_state=42)
train_set_y = dataset.loc[train_set_X.index]['endpoint'].values
test_set_y = dataset.loc[test_set_X.index]['endpoint'].values

In [2]:
from sklearn import pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
pipeline = pipeline.Pipeline([
        ('scaling', MinMaxScaler()),
        ('std_scaler', StandardScaler()),
    ])
train_X_prepared = pipeline.fit_transform(train_set_X)
test_X_prepared = pipeline.transform(test_set_X)

# Random forest

In [3]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 100, num = 5)]

# Number of features to consider at every split
max_features = ['auto', None]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [4]:
from sklearn.ensemble import RandomForestRegressor
# Use the random grid to search for best hyperparameters
# First create the base model to tune

rf = RandomForestRegressor()

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model

rf_random.fit(train_X_prepared, train_set_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.5min finished
D:\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [5]:
from sklearn.metrics import r2_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    MAE = np.mean(errors)
    r2 = r2_score(test_labels, predictions)
    print('Model Performance')
    print('MAE: {:0.2f}.'.format(MAE))
    print('r2 = {:0.2f}.'.format(r2))
    
    return MAE


base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_X_prepared, train_set_y)
base_accuracy = evaluate(base_model, test_X_prepared, test_set_y)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_X_prepared,  test_set_y)

Model Performance
MAE: 0.65.
r2 = 0.51.
Model Performance
MAE: 0.62.
r2 = 0.53.


In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 20, 30, 40, 50],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2],
    'n_estimators': [5, 100, 150, 200, 250, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [4]:
# Fit the grid search to the data
grid_search.fit(train_X_prepared, train_set_y)
grid_search.best_params_

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed: 11.8min finished


{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [5]:
from sklearn.metrics import r2_score
import numpy as np
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    MAE = np.mean(errors)
    r2 = r2_score(test_labels, predictions)
    print('Model Performance')
    print('MAE: {:0.2f}.'.format(MAE))
    print('r2 = {:0.2f}.'.format(r2))
    
    return MAE

In [6]:
best_grid = grid_search.best_estimator_
df = pd.DataFrame(test_X_prepared)
df = df.fillna(df.mean())
test_X_prepared = df.values
grid_accuracy = evaluate(best_grid, test_X_prepared, test_set_y)

Model Performance
MAE: 0.62.
r2 = 0.53.


In [7]:
# best_grid = grid_search.best_estimator_
from sklearn.externals import joblib
joblib.dump(best_grid, "EB_rf_model_0806.pkl")
#my_model_loaded = joblib.load("my_model.pkl") 

D:\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['EB_rf_model_0806.pkl']

# SVM

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# Create the parameter grid based on the results of random search 
param_grid = {
    'kernel': ['rbf'],
    'gamma': [1e-2, 1e-3],
    'C': [1,10]}
# Create a based model
svm = SVR()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svm, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [9]:
# Fit the grid search to the data
grid_search.fit(train_X_prepared, train_set_y)
grid_search.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  20 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.5s finished


{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [10]:
best_grid = grid_search.best_estimator_
df = pd.DataFrame(test_X_prepared)
df = df.fillna(df.mean())
test_X_prepared = df.values
grid_accuracy = evaluate(best_grid, test_X_prepared, test_set_y)

Model Performance
MAE: 0.65.
r2 = 0.45.


In [11]:
best_grid = grid_search.best_estimator_
from sklearn.externals import joblib
joblib.dump(best_grid, "EB_svm_model_0806.pkl")

['EB_svm_model_0806.pkl']